In [1]:
! pip install edinet-xbrl

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/41.5 kB ? eta -:--:--
     ---------------------------------------- 41.5/41.5 

In [ ]:
# ファイルパスのパターンにワイルドカードを使うので必要
import glob
# xbrl解析に必要
from edinet_xbrl.edinet_xbrl_parser import EdinetXbrlParser
# 不必要なタグ・スペースを除去するのに必要
import re

In [23]:
def Business_risk(file_name_list):
    #パーサのインスタンス化
    parser = EdinetXbrlParser()

    for file_name in file_name_list:
        xbrl_path = f"influencer_marketing_data_set/{file_name}/S*/XBRL/PublicDoc/*.xbrl"


        xbrl_path_all = glob.glob(xbrl_path)
        print(xbrl_path_all)

        xbrl_path = glob.glob(xbrl_path)[0]

        edinet_xbrl_object = parser.parse_file(xbrl_path)

        # 事業等のリスクをとってみる
        # <jpcrp_cor:BusinessRisksTextBlock contextRef="FilingDateInstant">
        # ってところに事業等のリスクがあるので
        key='jpcrp_cor:BusinessRisksTextBlock'
        context_ref='FilingDateInstant'
        data = edinet_xbrl_object.get_data_by_context_ref(key, context_ref)
        text_data = data.get_value()

        # 空白（\s）除去
        text_data = re.sub('\s','',text_data)
        # 任意の文字数のタグ（<.*?>）の除去
        text_data = re.sub('<.*?>','',text_data)

        # テキストデータの保存
        text_file = open(f'Business_risk/{file_name}_Business_risk.txt', 'w', encoding='UTF-8')
        text_file.write(text_data)
        text_file.close()

<>:25: SyntaxWarning: invalid escape sequence '\s'
<>:25: SyntaxWarning: invalid escape sequence '\s'
C:\Users\rhamano\AppData\Local\Temp\ipykernel_8360\134517061.py:25: SyntaxWarning: invalid escape sequence '\s'
  text_data = re.sub('\s','',text_data)


In [24]:
file_name_list = ["anymindgroup_2023","cyberbuzz_2023","trenders_2023"]
Business_risk(file_name_list)

['influencer_marketing_data_set/anymindgroup_2023\\S100QHY2\\XBRL\\PublicDoc\\jpcrp030000-asr-001_E37466-000_2022-12-31_01_2023-03-30.xbrl']
['influencer_marketing_data_set/cyberbuzz_2023\\S100SGQG\\XBRL\\PublicDoc\\jpcrp030000-asr-001_E35089-000_2023-09-30_01_2023-12-14.xbrl']
['influencer_marketing_data_set/trenders_2023\\S100R4DE\\XBRL\\PublicDoc\\jpcrp030000-asr-001_E26933-000_2023-03-31_01_2023-06-26.xbrl']
